In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.datasets import Planetoid, LRGBDataset
import scipy.sparse as sp
import numpy as np
from numpy.linalg import eigh

In [ ]:
# Laplacian Positional Encoding (LapPE)
def laplacian_positional_encoding(edge_index, num_nodes, k=10):
    edge_index = edge_index.cpu().numpy()
    row, col = edge_index[0], edge_index[1]
    adj = sp.coo_matrix((np.ones(len(row)), (row, col)), shape=(num_nodes, num_nodes))
    degree = sp.diags(adj.sum(axis=1).A1)
    
    # Compute Laplacian
    laplacian = degree - adj
    eigenvalues, eigenvectors = eigh(laplacian.toarray())
    
    # Return top k eigenvectors as LapPE
    return torch.tensor(eigenvectors[:, :k], dtype=torch.float)

In [ ]:
from torch_geometric.utils import to_scipy_sparse_matrix
from scipy.sparse.csgraph import laplacian as cs_laplacian
from scipy.sparse.linalg import eigsh
from numpy.linalg import eigh
import numpy as np
import torch

def compute_lap_pe(data, pe_dim=10, use_signnet=False):
    # Compute Laplacian and eigenvectors
    adj = to_scipy_sparse_matrix(data.edge_index, num_nodes=data.num_nodes).astype(float)
    lap = cs_laplacian(adj, normed=False)
    
    try:
        eigvals, eigvecs = eigsh(lap, k=pe_dim + 1, which='SM')
    except RuntimeError:
        eigvals, eigvecs = eigh(lap.toarray())
    
    pe = eigvecs[:, 1:pe_dim + 1]  # Exclude first eigenvector

    # Handle sign ambiguity (if not using SignNet, apply random sign flips)
    if not use_signnet:
        pe *= np.random.choice([-1, 1], size=(pe.shape[1],))

    # Add to data object
    data.lap_pe = torch.tensor(pe, dtype=torch.float)
    return data


In [3]:
class SignNet(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super(SignNet, self).__init__()
        self.linear = torch.nn.Linear(in_features, out_features)

    def forward(self, x):
        return self.linear(x) + self.linear(-x)

In [4]:
def rwse(adj_matrix, k=10):
    powers = [adj_matrix**i for i in range(1, k + 1)]
    rwse_values = torch.stack([torch.trace(power).item() for power in powers], dim=0)
    return rwse_values

In [31]:
from torch_geometric.nn import TransformerConv
from torch_geometric.nn import global_mean_pool

class GraphTransformer(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, heads=8):  # Use 8 heads as per the task
        super(GraphTransformer, self).__init__()
        self.transformer = TransformerConv(in_channels, hidden_channels, heads=heads)
        self.out_channels = hidden_channels * heads  # 64 * 8 = 512

    def forward(self, x, edge_index):
        return F.relu(self.transformer(x, edge_index))


class GraphTransformerClassifier(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes, task="node"):
        super(GraphTransformerClassifier, self).__init__()
        self.transformer = GraphTransformer(in_channels, hidden_channels, heads=8)
        self.reduce_dim = torch.nn.Linear(hidden_channels * 8, hidden_channels)  # 512 -> 64
        self.task = task
        self.pool = global_mean_pool if task == "graph" else None
        self.fc = torch.nn.Linear(hidden_channels, num_classes)

    def forward(self, x, edge_index, batch=None):
        x = self.transformer(x, edge_index)
        x = self.reduce_dim(x)  # Reduce dimension from 512 to 64
        if self.task == "graph":
            x = self.pool(x, batch)  # Global mean pooling for graph-level tasks
        return self.fc(x)

In [ ]:
# Example with Cora Dataset
cora_dataset = Planetoid(root='./dataset/cora', name='Cora')
cora_data = cora_dataset[0]

print(f"Original features: {cora_data.num_node_features}")  # Should be 1433

# Compute LapPE for Cora dataset
lap_pe = laplacian_positional_encoding(cora_data.edge_index, cora_data.num_nodes, k=10)
cora_data.x = torch.cat([cora_data.x, lap_pe], dim=1)  # Concatenate LapPE with original features

print(f"Laplacian Positional Encoding shape: {lap_pe.shape}")  # Should be [2708, 10]
print(f"Concatenated features shape: {cora_data.x.shape}")  # Should be [2708, 1443]
print(f"New features: {cora_data.num_node_features}")  # Should be 1433

c:\Users\Akshat\anaconda3\envs\lua_torch\lib\site-packages\torch_geometric\data\dataset.py:238: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  if osp.exists(f) and torch.load

Original features: 1433
Laplacian Positional Encoding shape: torch.Size([2708, 10])
Concatenated features shape: torch.Size([2708, 1443])
Original features: 1443


In [44]:
def train(model, loader, optimizer, criterion, task="node"):
    model.train()
    total_loss = 0

    for batch in loader:  # Batch comes from DataLoader
        optimizer.zero_grad()

        if task == "node":
            out = model(batch.x, batch.edge_index)
            loss = criterion(out[batch.train_mask], batch.y[batch.train_mask])
        else:  # Graph classification
            out = model(batch.x, batch.edge_index, batch.batch)
            loss = criterion(out, batch.y)

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(loader)


def evaluate(model, loader, criterion, task="node"):
    model.eval()
    correct = 0
    total_loss = 0

    # Handle single graph datasets
    if len(loader) == 1:  # Single graph case
        graph_data = loader.dataset[0]  # Access the single graph
        total_nodes = graph_data.num_nodes if task == "node" else 1  # Total nodes for node classification
    else:
        total_nodes = sum(batch.num_nodes for batch in loader.dataset)

    for batch in loader:
        with torch.no_grad():
            if task == "node":
                out = model(batch.x, batch.edge_index)
                pred = out.argmax(dim=1)
                correct += (pred[batch.test_mask] == batch.y[batch.test_mask]).sum().item()
                loss = criterion(out[batch.test_mask], batch.y[batch.test_mask])
            else:  # Graph classification
                out = model(batch.x, batch.edge_index, batch.batch)
                pred = out.argmax(dim=1)
                correct += (pred == batch.y).sum().item()
                loss = criterion(out, batch.y)

            total_loss += loss.item()

    accuracy = correct / total_nodes
    return total_loss / len(loader), accuracy


In [53]:
cora_model = GraphTransformerClassifier(
    in_channels=cora_data.num_node_features,
    hidden_channels=64,
    out_channels=32,
    num_classes=cora_dataset.num_classes,
    task="node"
).to('cuda')

In [54]:
# Initialize model, optimizer, and loss function
optimizer = torch.optim.AdamW(cora_model.parameters(), lr=0.0001, weight_decay=1e-4)
criterion = torch.nn.CrossEntropyLoss()

from torch_geometric.loader import DataLoader

cora_loader = DataLoader([cora_data.cuda()], batch_size=1)  # Single graph wrapped in DataLoader

# Training loop
for epoch in range(500):
    train_loss = train(cora_model, cora_loader, optimizer, criterion, task="node")
    test_loss, test_accuracy = evaluate(cora_model, cora_loader, criterion, task="node")

    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


Epoch 1, Train Loss: 1.9498, Test Loss: 1.9174, Test Accuracy: 0.1178
Epoch 2, Train Loss: 1.9428, Test Loss: 1.9161, Test Accuracy: 0.1178
Epoch 3, Train Loss: 1.9358, Test Loss: 1.9147, Test Accuracy: 0.1178
Epoch 4, Train Loss: 1.9289, Test Loss: 1.9133, Test Accuracy: 0.1178
Epoch 5, Train Loss: 1.9220, Test Loss: 1.9119, Test Accuracy: 0.1178
Epoch 6, Train Loss: 1.9151, Test Loss: 1.9105, Test Accuracy: 0.1178
Epoch 7, Train Loss: 1.9082, Test Loss: 1.9090, Test Accuracy: 0.1178
Epoch 8, Train Loss: 1.9013, Test Loss: 1.9076, Test Accuracy: 0.1178
Epoch 9, Train Loss: 1.8944, Test Loss: 1.9060, Test Accuracy: 0.1178
Epoch 10, Train Loss: 1.8874, Test Loss: 1.9045, Test Accuracy: 0.1178
Epoch 11, Train Loss: 1.8804, Test Loss: 1.9029, Test Accuracy: 0.1178
Epoch 12, Train Loss: 1.8732, Test Loss: 1.9012, Test Accuracy: 0.1178
Epoch 13, Train Loss: 1.8659, Test Loss: 1.8995, Test Accuracy: 0.1182
Epoch 14, Train Loss: 1.8585, Test Loss: 1.8978, Test Accuracy: 0.1182
Epoch 15, Train

In [8]:
# LapPE and SignNet
lap_pe = laplacian_positional_encoding(data.edge_index, data.num_nodes, k=10).cuda()
sign_net = SignNet(lap_pe.shape[1], 64).cuda()
lap_pe_transformed = sign_net(lap_pe)

# Graph Transformer
graph_transformer = GraphTransformer(in_channels=64, out_channels=32).cuda()
x_transformed = graph_transformer(lap_pe_transformed, data.edge_index.cuda())

print("Transformed embeddings shape:", x_transformed.shape)

Transformed embeddings shape: torch.Size([2708, 256])
